# Interpretation Inspection

Display interpreter input alongside output, highlight duplicates removed and conflicts resolved with reasoning.

In [ ]:
import sys
import json
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
from IPython.display import Markdown, display
from src.config import BASELINE_FILE, EXTRACTIONS_DIR, OUTPUT_DIR
from src.loader import load_emr_file
from src.schemas import ExtractorOutput, InterpreterOutput
from src.interpreter import interpret

## Load Extraction Results

In [ ]:
extractor_outputs: list[ExtractorOutput] = []
for p in sorted(EXTRACTIONS_DIR.glob("*.json")):
    data = json.loads(p.read_text(encoding="utf-8"))
    extractor_outputs.append(ExtractorOutput.model_validate(data))

baseline_sheets = load_emr_file(BASELINE_FILE)
print(f"Loaded {len(extractor_outputs)} extraction files")
print(f"Loaded {len(baseline_sheets)} baseline sheets")

## Input Summary

In [ ]:
total_findings = 0
for eo in extractor_outputs:
    count = len(eo.findings)
    total_findings += count
    print(f"  {eo.sheet_name}: {count} findings")
print(f"\nTotal input findings: {total_findings}")

## Run Interpreter

In [ ]:
interpreter_output = await interpret(extractor_outputs, baseline_sheets)

print(f"Reconciled findings: {len(interpreter_output.reconciled_findings)}")
print(f"Conflicts resolved:  {len(interpreter_output.conflicts_resolved)}")
print(f"Duplicates removed:  {interpreter_output.duplicates_removed}")
print(f"Input findings:      {interpreter_output.metadata.total_input_findings}")
print(f"Output findings:     {interpreter_output.metadata.total_output_findings}")

## Reconciled Findings

In [ ]:
rows = []
for i, rf in enumerate(interpreter_output.reconciled_findings, 1):
    sources = ", ".join(rf.sources)
    note = rf.resolution_note or "—"
    content_preview = rf.content[:100] + ("…" if len(rf.content) > 100 else "")
    rows.append(f"| {i} | {rf.datetime or '—'} | {content_preview} | {sources} | {note} |")

table = (
    "| # | Datetime | Content | Sources | Resolution Note |\n"
    "|---|----------|---------|---------|-----------------|\n"
    + "\n".join(rows)
)
display(Markdown(table))

## Conflict Resolutions

In [ ]:
if not interpreter_output.conflicts_resolved:
    display(Markdown("*No conflicts detected.*"))
else:
    for i, cr in enumerate(interpreter_output.conflicts_resolved, 1):
        md = (
            f"### Conflict {i}\n\n"
            f"**Description:** {cr.description}\n\n"
            f"**Sources:** {', '.join(cr.sources)}\n\n"
            f"**Resolution:** {cr.resolution}"
        )
        display(Markdown(md))

## Save Results

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
interp_path = OUTPUT_DIR / "interpretation.json"
interp_path.write_text(
    json.dumps(interpreter_output.model_dump(), ensure_ascii=False, indent=2),
    encoding="utf-8",
)
print(f"Saved: {interp_path}")